In [1]:
#import os
#import json
import numpy as np
import pandas as pd
from google.colab import drive
#from google.cloud import firestore
import firebase_admin
from firebase_admin import firestore, credentials

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#JSON_KEY_FILE = '/content/drive/My Drive/firestore-446822-ac05de749f4a.json'

#JSON_KEY_FILE = "/content/drive/My Drive/service-pa-79ac5-0f0babe13506.json"

JSON_KEY_FILE = "/content/drive/My Drive/csg3101-service-providing-app-firebase-adminsdk-7xpfw-f39bc603db.json"

In [4]:
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_KEY_FILE

# Initialize Firebase Admin SDK
cred = credentials.Certificate(JSON_KEY_FILE)
firebase_admin.initialize_app(cred)

In [5]:
# Initialize Firestore DB
db = firestore.client()

In [6]:
# Define your collection name
collection_name = 'bookings'

# Get all documents from the collection
docs = db.collection(collection_name).stream()

In [7]:
# Define the number of documents to read
#num_docs = 1

# Prepare the list to hold all the document data
data = []

# Loop through each document and append it to the list
for i, doc in enumerate(docs):
    #if i == num_docs:  # Stop after reading the specified number of documents
    #  break
    doc_dict = doc.to_dict()  # Convert document to a dictionary
    doc_dict['bookingId'] = doc.id   # Add document ID as a field (optional)
    data.append(doc_dict)

In [8]:
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

In [9]:
# Display the result
print(df.head())

           dateOfServiceCompletion  serviceDuration              customerId  \
0 2025-01-19 11:04:58.804000+00:00               39  bFPdtzu11Oi0f92EAcjqmg   
1 2025-01-19 11:04:58.367000+00:00                6  clWLI5OZP2ad25ugMVI8gg   
2 2025-01-19 11:04:58.562000+00:00               26  Jha0USGDMefGFRLik_xFQg   
3 2025-01-19 11:04:56.707000+00:00               13  OyoGAe7OKpv6SyGZT5g77Q   
4 2025-01-19 11:05:00.641000+00:00               25  xbybLiQockAzC4xAlzFrGg   

   chargePerHour BookingCancellationReason                    dateOfBooking  \
0           2530                      None 2025-01-19 11:04:58.804000+00:00   
1           2000                      None 2025-01-19 11:04:58.367000+00:00   
2           2000                      None 2025-01-19 11:04:58.562000+00:00   
3           2000                      None 2025-01-19 11:04:56.707000+00:00   
4           2000                      None 2025-01-19 11:05:00.641000+00:00   

      status              employeeId  serviceType 

In [10]:
print(len(df)) # total number of entries

100


In [11]:
# Remove unnecessary column from the original DataFrame
df = df.drop(columns=['dateOfServiceCompletion', 'customerId', 'BookingCancellationReason', 'dateOfBooking', 'status', 'employeeId', 'serviceType', 'dateOfBookingCancellation'])

In [12]:
# Display the result
print(df.head())

   serviceDuration  chargePerHour             bookingId
0               39           2530  0yEyFDjZ1EFS5QRvyOC4
1                6           2000  1zz51pDrGnIktZd89o1D
2               26           2000  2jUz16XvwHLkW2Feq3YO
3               13           2000  2jqs3ejHcgwH2gKGFdsK
4               25           2000  3gv5xYN8L7bFRPZ38Kiq


In [13]:
# Create the new column 'totalCharge' as the product of serviceDuration and chargePerHour
df['paymentAmount'] = df['serviceDuration'] * df['chargePerHour']

In [14]:
# Display the result
print(df.head())

   serviceDuration  chargePerHour             bookingId  paymentAmount
0               39           2530  0yEyFDjZ1EFS5QRvyOC4          98670
1                6           2000  1zz51pDrGnIktZd89o1D          12000
2               26           2000  2jUz16XvwHLkW2Feq3YO          52000
3               13           2000  2jqs3ejHcgwH2gKGFdsK          26000
4               25           2000  3gv5xYN8L7bFRPZ38Kiq          50000


In [15]:
# Remove serviceDuration and chargePerHour column from the original DataFrame
df = df.drop(columns=['serviceDuration', 'chargePerHour'])

In [16]:
# Display the result
print(df.head())

              bookingId  paymentAmount
0  0yEyFDjZ1EFS5QRvyOC4          98670
1  1zz51pDrGnIktZd89o1D          12000
2  2jUz16XvwHLkW2Feq3YO          52000
3  2jqs3ejHcgwH2gKGFdsK          26000
4  3gv5xYN8L7bFRPZ38Kiq          50000


In [17]:
# Add a new 'paymentMethod' column
df['paymentMethod'] = 'credit/debit card'

# Display the result
print(df.head())

              bookingId  paymentAmount      paymentMethod
0  0yEyFDjZ1EFS5QRvyOC4          98670  credit/debit card
1  1zz51pDrGnIktZd89o1D          12000  credit/debit card
2  2jUz16XvwHLkW2Feq3YO          52000  credit/debit card
3  2jqs3ejHcgwH2gKGFdsK          26000  credit/debit card
4  3gv5xYN8L7bFRPZ38Kiq          50000  credit/debit card


In [18]:
# Add a new 'paymentDate' column
df['paymentDate'] = firestore.SERVER_TIMESTAMP

# Display the result
print(df.head())

              bookingId  paymentAmount      paymentMethod  \
0  0yEyFDjZ1EFS5QRvyOC4          98670  credit/debit card   
1  1zz51pDrGnIktZd89o1D          12000  credit/debit card   
2  2jUz16XvwHLkW2Feq3YO          52000  credit/debit card   
3  2jqs3ejHcgwH2gKGFdsK          26000  credit/debit card   
4  3gv5xYN8L7bFRPZ38Kiq          50000  credit/debit card   

                                         paymentDate  
0  Sentinel: Value used to set a document field t...  
1  Sentinel: Value used to set a document field t...  
2  Sentinel: Value used to set a document field t...  
3  Sentinel: Value used to set a document field t...  
4  Sentinel: Value used to set a document field t...  


In [19]:
# Convert DataFrame to JSON
processed_data = df.to_dict(orient='records')

In [20]:
print(processed_data[0])

{'bookingId': '0yEyFDjZ1EFS5QRvyOC4', 'paymentAmount': 98670, 'paymentMethod': 'credit/debit card', 'paymentDate': Sentinel: Value used to set a document field to the server timestamp.}


In [21]:
print(len(processed_data))

100


In [22]:
# Firestore collection name
collection_name = "payments"

In [23]:
# Upload processed data to Firestore
for doc in processed_data:

  # Reference to your Firestore collection
  collection_ref = db.collection(collection_name)

  # Add the document with an auto-generated ID
  doc_ref = collection_ref.add(doc)

In [24]:
print("Processed data uploaded successfully!")

Processed data uploaded successfully!
